# Extraire-Données-CASSIOPE
***

**Tutoriel :** Ce tutoriel explique comment extraire les données de CASSIOPE avec les plateformes de l'Université de Calgary.    
**Mission et Instrument :** CASSIOPE    
**Objectif astronomique :** Étudier l'ionosphère    
**Outil principal :** eDex Web    
**Exigences du système :** Accès à Internet  
**Niveau du tutoriel :** Intermédiaire    

Il y a deux endroits où on peut avoir accès aux données e-POP de CASSIOPE. Ils sont résumés [ici](https://epop.phys.ucalgary.ca/data/) avec plus d'informations et outils. Merci à l'Université de Calgary pour leur aide et conseils dans l'élaboration de ce tutoriel.

## Accès HTTP
***

[L'accès HTTP](https://epop-data.phys.ucalgary.ca/) est utile pour naviguer les fichiers, vérifier la disponibilité des données et regarder les fichiers par instrument. Les données sont classées par jour.

## eDex Web
***

[eDex Web](https://edex.phys.ucalgary.ca/) (e-POP Data Explorer website) est une API qui permet de trouver et de télécharger des données par date, contraintes géophysiques et e-POP. Il y a deux façons d'extraire les données :

1. Vous pouvez créer votre requête en ligne, obtenir les résultats et recevoir les fichiers par courriel.
2. Vous pouvez utiliser un script python pour envoyer votre requête et vous recevrez les fichiers par courriel. 

Pour plus d'informations, il y a de la [documentation](https://epop.phys.ucalgary.ca/data-handbook/edex-api-documentation/) sur le site web de l'Université de Calgary.

### En ligne

Le eDex est organisé avec sept onglets : *Output Product Selection*, *Date Constraints*, *Geophysical Constraints*, *e-POP Constraints*, *Planned Experiment Constraints*, *Query Builder* et *Results*. Pour naviguer entre eux, il vaut mieux cliquer sur l'onglet que sur le bouton *Next Step...*. Voyons ce à quoi correspond chaque onglet.

#### Output Product Selection

Lorsque vous chargez l'eDex pour la première fois, vous vous trouvez dans la section *Output Product Selection* où vous pouvez sélectionner le type de fichiers que vous souhaitez télécharger. Pour mieux comprendre les fichiers, vous pouvez consulter le [guide](https://epop.phys.ucalgary.ca/data-handbook/) ou l'[accès HTTP](https://epop-data.phys.ucalgary.ca/).

#### Date Constraints

L'onglet *Date Constraints* est explicite. Vous sélectionnez la période pour laquelle vous souhaitez obtenir les fichiers. Une fois que vous avez choisi vos dates de début et de fin, cliquez sur *Add Constraint*. Vous pouvez ajouter autant de périodes que vous le souhaitez. Vous pouvez également mettre à jour ou supprimer une contrainte en la sélectionnant à droite de votre écran sous *Constraints* et en cliquant sur le bouton correspondant.

#### Geophysical Constraints

Cet onglet vous permet de filtrer les données en fonction de certaines métriques. Pour voir ce qui se cache derrière le ..., vous devez sélectionner une option dans la section au-dessus.

#### e-POP Constraints

La majorité des *e-POP Constraints* sont spécifiques à chaque instrument. Vous pouvez également ajouter une contrainte concernant la position ou l'attitude de l'engin spatial. Le champ *Has Data* ne semble pas avoir d'influence sur les résultats dans la plupart des cas.

#### Planned Experiment Constraints

Cet onglet est utilisé pour spécifier une expérience réalisée sur les données. Par exemple, avec *Point of Interest* -> *Fort McMurray Fire* et avec les dates de cet événement, vous verrez des données relatives à ces incendies en 2016.

#### Query Builder

Cet onglet est probablement l'un des plus importants et celui qui a le plus d'influence sur votre extraction. Avec les contraintes que vous avez ajoutées à droite et leur ID, vous devez construire la logique entre elles. Par exemple, si je voulais comparer les données de novembre 2017 (1) et les données de novembre 2018 (2), j'aurais besoin de l'opérateur OR (1 OR 2). Une fois que vous avez écrit la logique, il est important de cliquer sur *Set Query*.

#### Results

Il s'agit de la dernière étape avant le téléchargement de vos données. Il vous suffit de cliquer sur *Fetch Results...*, de consulter la liste des fichiers (et de décocher des fichiers si nécessaire), de saisir votre adresse courriel et de cliquer sur *Download Results...*. Vous recevrez les données par e-mail quelques minutes plus tard dans un fichier .zip.

### Python Script

Le script suivant est une version simplifiée de celui disponible sur le [site web](https://epop.phys.ucalgary.ca/data-handbook/edex-api-documentation/) de l'Université de Calgary écrit par Troy Kachor. Il s'agit d'une demande à eDex pour les fichiers HDF5 du 1er mai 2019 de l'instrument FAI. Nous utiliserons ces fichiers dans l'exemple FAI.

In [1]:
import requests

# URL pour rechercher les données
API_fetch = "https://api.edex.phys.ucalgary.ca/api_fetch_data"

# Là où les noms des fichiers seront stockés
filename_list = []

# Construire la requête (ne pas oublier de changer l'adresse courriel)
PARAMS = {
            'product': ['fai-lv1-hdf5-images'],
            'date': [['2019-05-01', '2019-05-01']],
            'fai': ['has-data'],
            'email': "youremail@example.ca",
            'result_filenames': filename_list,
            'query-logic': ' ( 1 AND 2 ) '
          }

# Obtenir la liste des fichiers
r = requests.get(url=API_fetch, json=PARAMS)

# Assigner la liste de fichiers à filename_list
filename_list = r.json()["result_filenames"]

# Imprimer la liste de fichier
print("Liste de fichiers correspondants à votre demande :")
print(filename_list)

# Confirme que ces fichiers sont ceux à télécharger
# input("Appuyez sur Enter pour télécharger ou ctrl-c pour annuler...")

# URL pour télécharger les fichiers
API_get = "https://edex.phys.ucalgary.ca/api_prepare_report"

# Mettre à jour la requête avec la liste de fichiers
PARAMS['result_filenames'] = filename_list

# Envoyer la demande
status = requests.post(url=API_get, json=PARAMS)

# Message de statut
print('Status de votre demande :')
print(status)
print(status.text)

Liste de fichiers correspondants à votre demande :
['FAI_lv1_h5_20190501_150534_151306_6.0.0.zip   540.57MB', 'FAI_lv1_h5_20190501_164634_165406_6.0.0.zip   542.65MB', 'FAI_lv1_h5_20190501_182724_183456_6.0.0.zip   543.11MB']
Status de votre demande :
<Response [200]>
Preparing report... You will receive an email from esoc@phys.ucalgary.ca shortly.


Après avoir exécuté ce script, vous devrez attendre quelques minutes avant de recevoir un courriel contenant vos données. Pour plus d'informations sur l'utilisation de ce type de script et des paramètres, consultez la [Documentation de l'API](https://epop.phys.ucalgary.ca/data-handbook/edex-api-documentation/).

## Extraire les fichiers d'un dossier compressé
***

Lorsque vous recevez un lien de téléchargement par courriel de l'Université de Calgary avec vos fichiers, vous téléchargez un dossier compressé (.zip). Voici comment extraire les fichiers d'un dossier compressé avec une version récente de Windows.

1. Allez dans le répertoire où se trouve votre dossier compressé.
2. Faites un clic droit sur le dossier compressé.
3. Cliquez sur "Extraire tout...".
4. Choisissez l'emplacement de vos fichiers.
5. Cliquez sur "Extraire".

Selon le nombre de fichiers que vous avez téléchargés, cela peut prendre un certain temps.